In [144]:
from valurap2 import commands
import imp
imp.reload(commands)

p = commands.S3GPort()
p.S3G_OUTPUT(63, 33)
p.S3G_OUTPUT(7, 0x00000000)
p.S3G_INPUT(63)

33

In [2]:
s_t = 0.001
f = 50000000
sb = 40

n = 400 * 32
t = 1

v = n * (2 ** sb) / f / t
v1 = v * 2
n_s = t / s_t
a = v1 / n_s
dt = f * s_t
v, a, dt, n_s

(281474976.710656, 562949.9534213119, 50000.0, 1000.0)

In [117]:
import time
a = [
    p.BUF_OUTPUT(0, 50000),        # step time 0.001s
    p.BUF_OUTPUT(2, 40),           # step_bit = 40
    p.BUF_OUTPUT(3, 100),          # pre_n
    p.BUF_OUTPUT(4, 400),          # pulse_n
    p.BUF_OUTPUT(5, 500),          # post_n
    p.BUF_OUTPUT(7, 0x80818080),   # enable 1 and 2 motors on SG0
    p.BUF_OUTPUT(13, 500000),       # ES_TIMEOUT 10ms

    p.BUF_OUTPUT(1, 20000),              # steps_val = 2000 (2 seconds)
    p.BUF_PARAM_ADDR(0),
    p.BUF_PARAM_WRITE_LO(0, 3),           # CH0.STATUS=3 (enable + target_v)
    p.BUF_PARAM_WRITE_LO(3, 0),           # CH0.V_OUT=0
    p.BUF_PARAM_WRITE_LO(1, 500000),     # CH0.A
    p.BUF_PARAM_WRITE_LO(1, 0),           # CH0.J=0
    p.BUF_PARAM_WRITE_LO(1, 0),           # CH0.JJ=0
    p.BUF_PARAM_WRITE_LO(1, 20000000),  # CH0.TARGET_V
    p.BUF_PARAM_WRITE_LO(1, 500000),     # CH0.ABORT_A
    p.BUF_PARAM_WRITE_LO_NC(0),   # CH1.STATUS=0
    p.BUF_PARAM_WRITE_LO_NC(0),   # CH2
    p.BUF_PARAM_WRITE_LO_NC(0),   # CH3
    p.BUF_PARAM_WRITE_LO_NC(0),   # CH4
    p.BUF_PARAM_WRITE_LO_NC(0),   # CH5
    p.BUF_PARAM_WRITE_LO_NC(0),   # CH6
    p.BUF_PARAM_WRITE_LO_NC(0),   # CH7
    p.BUF_STB(p.STB_ASG_START),   # Start ASG cycles
    
    p.BUF_WAIT_ALL(p.INT_ASG_LOAD),  # Wait for APG ready for new segment
    p.BUF_CLEAR(p.INT_ASG_LOAD),     # Clear pending int
    
    p.BUF_OUTPUT(1, 3000),           # steps_val = 3000 (3 seconds)
    p.BUF_PARAM_ADDR(0),
    p.BUF_PARAM_WRITE_LO(0, 3),          # CH0.STATUS=3
    p.BUF_PARAM_WRITE_LO(4, -500000),   # CH0.A
    p.BUF_PARAM_WRITE_LO(3, 5000000),  # CH0.TARGET_V
    p.BUF_STB(p.STB_ASG_LOAD_DONE),  # Inform ASG, next segment setup is done

    p.BUF_WAIT_ALL(p.INT_ASG_LOAD),
    p.BUF_CLEAR(p.INT_ASG_LOAD),
    
    p.BUF_OUTPUT(1, 1000),      # steps_val = 1000 (1 second)
    p.BUF_PARAM_ADDR(0),
    p.BUF_PARAM_WRITE_LO(0, 3),          # CH0.STATUS=3
    p.BUF_PARAM_WRITE_LO(4, -500000),   # CH0.A
    p.BUF_PARAM_WRITE_LO(3, 0),          # CH0.TARGET_V=0
    p.BUF_STB(p.STB_ASG_LOAD_DONE),
    
    p.BUF_WAIT_ALL(p.INT_ASG_LOAD),
    p.BUF_CLEAR(p.INT_ASG_LOAD),

    p.BUF_OUTPUT(1, 0),         # steps_val = 0 - End of path
    p.BUF_STB(p.STB_ASG_LOAD_DONE),
    p.BUF_WAIT_ALL(p.INT_ASG_DONE),  # Wait for ASG done
    p.BUF_CLEAR(p.INT_ASG_DONE),
    
    p.BUF_OUTPUT(7, 0x00000000), # disable 1 and 2 motors
    p.BUF_DONE(0)                # End of program for BE
]

p.S3G_STB(p.STB_BE_ABORT)      # Reset BE and ASG just in case
p.S3G_STB(p.STB_ASG_ABORT)
p.S3G_WRITE_FIFO(*a)           # Send program into FIFO
p.S3G_STB(p.STB_BE_START)      # Start execution
p.S3G_STB(p.STB_ES_UNLOCK)

oa, ob, oc, od1, od2 = None, None, None, -1, -1
pd1 = -1
pd2 = -1
while True:
    a = p.S3G_INPUT(p.IN_BE_STATUS)
    b = p.S3G_INPUT(p.IN_APG_STATUS)
    c = p.S3G_INPUT(p.IN_FIFO_DATA_COUNT)
    d1 = p.S3G_INPUT(p.IN_MOTOR1_HOLD_X)
    d2 = p.S3G_INPUT(p.IN_MOTOR2_HOLD_X)
    
    if (a, b, c, d1, d2) != (oa, ob, oc, od1, od2):
        print(a,b,c,d, od1 - d1, od2 - d2)
        if (d1, d2) != (od1, od2):
            es = p.S3G_INPUT(p.IN_ES_STATUS)
            if es & 2:
                print("es1", es, d1 - pd1)
                pd1 = d1
                p.S3G_STB(p.STB_ES_UNLOCK)
            if es & 32:
                print("es1", es, d2 - pd2)
                pd2 = d2
                p.S3G_STB(p.STB_ES_UNLOCK)
        
    (oa, ob, oc, od1, od2) = (a,b,c,d1,d2) 
    if a & 1 == 0:
        break
    time.sleep(0.001)

9 1 23 304805 -363204 -361616
9 1 23 304805 0 -2704
es1 48 364320
9 1 23 304805 0 -112
es1 32 112
9 1 23 304805 -2803 0
es1 3 366007
9 1 23 304805 -79 0
es1 2 79
9 1 23 304805 0 -3088
es1 48 3088
9 1 23 304805 0 -112
es1 32 112
9 1 23 304805 -3121 0
es1 3 3121
9 1 23 304805 -79 0
es1 2 79
9 1 23 304805 0 -3088
es1 48 3088
9 1 23 304805 0 -112
es1 32 112
9 1 23 304805 -3121 0
es1 3 3121
9 1 23 304805 -79 0
es1 2 79
9 1 23 304805 0 -3088
es1 48 3088
9 1 23 304805 0 -112
es1 32 112
9 1 23 304805 -3121 0
es1 3 3121
9 1 23 304805 -79 0
es1 2 79
9 1 23 304805 0 -3088
es1 48 3088
9 1 23 304805 0 -112
es1 32 112
9 1 23 304805 -3121 0
es1 3 3121
9 1 23 304805 -79 0
es1 2 79
9 1 23 304805 0 -3088
es1 48 3088
9 1 23 304805 0 -112
es1 32 112
9 1 15 304805 0 0
9 1 15 304805 -3119 0
es1 3 3119
9 1 15 304805 -79 0
es1 2 79
9 1 7 304805 0 0
9 1 0 304805 0 0
0 0 0 304805 0 0


In [122]:
oa = None
p.S3G_STB(p.STB_ES_UNLOCK)

while True:
    a = p.S3G_INPUT(p.IN_ES_STATUS)
    
    if a != oa:
        print(a)
        p.S3G_STB(p.STB_ES_UNLOCK)
        
    oa = a 
    time.sleep(0.01)

0
48
16
32
0
48
16
32
0
3
1
2
0
3
1
2
0


KeyboardInterrupt: 

In [180]:
a = [
    p.BUF_OUTPUT(0, 50000),        # step time 0.001s
    p.BUF_OUTPUT(2, 40),           # step_bit = 40
    p.BUF_OUTPUT(3, 100),          # pre_n
    p.BUF_OUTPUT(4, 400),          # pulse_n
    p.BUF_OUTPUT(5, 500),          # post_n
    p.BUF_OUTPUT(7, 0x80988089),   # enable 1 and 2 motors on SG0
    p.BUF_OUTPUT(13, 500000),       # ES_TIMEOUT 10ms

    p.BUF_OUTPUT(1, 20000),              # steps_val = 20000 (20 seconds)
    p.BUF_PARAM_ADDR(0),
    p.BUF_PARAM_WRITE_LO(0, 3),           # CH0.STATUS=3 (enable + target_v)
    p.BUF_PARAM_WRITE_LO(3, 0),           # CH0.V_OUT=0
    p.BUF_PARAM_WRITE_LO(1, 500000),     # CH0.A
    p.BUF_PARAM_WRITE_LO(1, 0),           # CH0.J=0
    p.BUF_PARAM_WRITE_LO(1, 0),           # CH0.JJ=0
    p.BUF_PARAM_WRITE_LO(1, 30000000),  # CH0.TARGET_V
    p.BUF_PARAM_WRITE_LO(1, 500000),     # CH0.ABORT_A

    p.BUF_PARAM_WRITE_LO_NC(0, 3),           # CH0.STATUS=3 (enable + target_v)
    p.BUF_PARAM_WRITE_LO(3, 0),           # CH0.V_OUT=0
    p.BUF_PARAM_WRITE_LO(1, 500000),     # CH0.A
    p.BUF_PARAM_WRITE_LO(1, 0),           # CH0.J=0
    p.BUF_PARAM_WRITE_LO(1, 0),           # CH0.JJ=0
    p.BUF_PARAM_WRITE_LO(1, 20000000),  # CH0.TARGET_V
    p.BUF_PARAM_WRITE_LO(1, 500000),     # CH0.ABORT_A

    p.BUF_PARAM_WRITE_LO_NC(0),   # CH2
    p.BUF_PARAM_WRITE_LO_NC(0),   # CH3
    p.BUF_PARAM_WRITE_LO_NC(0),   # CH4
    p.BUF_PARAM_WRITE_LO_NC(0),   # CH5
    p.BUF_PARAM_WRITE_LO_NC(0),   # CH6
    p.BUF_PARAM_WRITE_LO_NC(0),   # CH7
    p.BUF_STB(p.STB_ASG_START),   # Start ASG cycles
    
    p.BUF_DONE(0)                # End of program for BE
]

p.S3G_STB(p.STB_BE_ABORT)      # Reset BE and ASG just in case
p.S3G_STB(p.STB_ASG_ABORT)
p.S3G_STB(p.STB_ES_UNLOCK)
p.S3G_CLEAR(0xffffffff)
p.S3G_OUTPUT(p.OUT_MSG_X_VAL, 0)
p.S3G_OUTPUT(p.OUT_MSG_CONFIG0, 0x82008200)
p.S3G_STB(p.STB_MSG_SET_X)
p.S3G_OUTPUT(p.OUT_MSG_CONFIG0, 0x80008000)
p.S3G_WRITE_FIFO(*a)           # Send program into FIFO
p.S3G_STB(p.STB_BE_START)      # Start execution

oa, ob, oc, od1, od2 = None, None, None, -1, -1
pd1 = -1
pd2 = -1
oes = None
while True:
    a = p.S3G_INPUT(p.IN_BE_STATUS)
    b = p.S3G_INPUT(p.IN_APG_STATUS)
    c = p.S3G_INPUT(p.IN_FIFO_DATA_COUNT)
    d1 = p.S3G_INPUT(p.IN_MOTOR1_HOLD_X)
    d2 = p.S3G_INPUT(p.IN_MOTOR2_HOLD_X)
    ints = p.S3G_INPUT(p.IN_PENDING_INTS)
    es = p.S3G_INPUT(p.IN_ES_STATUS)
    
    if (a, b, c, d1, d2, es) != (oa, ob, oc, od1, od2, oes):
        print(a,b,c,d, od1 - d1, od2 - d2, "{:b}".format(ints))
        if (d1, d2) != (od1, od2):
            if es & 32:
                print("es1", es, d1 - pd1)
                pd1 = d1
                p.S3G_STB(p.STB_ES_UNLOCK)
            if es & 2:
                print("es2", es, d2 - pd2)
                pd2 = d2
                p.S3G_STB(p.STB_ES_UNLOCK)
    if ints & (p.INT_ASG_DONE):
        print("stops are not reached")
        break
    if (ints & (p.INT_APG0_ABORT_DONE)) and (ints & (p.INT_APG1_ABORT_DONE)):
        print("Both stops are reached")
        cd1 = p.S3G_INPUT(p.IN_MOTOR1_X)
        cd2 = p.S3G_INPUT(p.IN_MOTOR2_X)
        print(pd1, cd1, cd1 - pd1, pd2, cd2, cd2 - pd2)
        break
        
    (oa, ob, oc, od1, od2, oes) = (a,b,c,d1,d2,es) 
    #if a & 1 == 0:
    #    break
    time.sleep(0.001)

0 1 0 304805 -3088 -58 1
0 1 0 304805 1617 0 1
es1 48 1471
0 1 0 304805 0 0 100000001
0 1 0 304805 -44 0 100000001
0 1 0 304805 0 -3039 1100000001
es2 19 3097
Both stops are reached
1470 1514 44 3096 3118 22
